In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

params = list(net.parameters())
print('lenth of params: ', len(params))
#print(params[0].size())  # conv1's .weight

input = torch.randn(1, 1, 32, 32)
output = net(input)
print('output: ',output)

target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
print('target: ',target)
criterion = nn.MSELoss()

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
print('old_loss: ',loss)
loss.backward()
optimizer.step()    # Does the update

output = net(input)
loss = criterion(output, target)
print('new_loss: ',loss)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
lenth of params:  10
output:  tensor([[-0.0421, -0.0464, -0.1360,  0.0839,  0.0631,  0.0382, -0.0134,  0.0907,
         -0.1348,  0.0729]], grad_fn=<AddmmBackward>)
target:  tensor([[ 0.0087,  0.5095, -0.3172, -0.0534, -0.4826, -1.7494, -1.0713,  0.1008,
          2.0132, -3.3757]])
old_loss:  tensor(2.1483, grad_fn=<MseLossBackward>)
new_loss:  tensor(2.1252, grad_fn=<MseLossBackward>)
